## Data Analysis

In [ ]:
import re

# Extraction of KJV

In [ ]:
import re
import pandas as pd

# File paths
input_file = "KJV_reformatted.txt"
output_file = "KJV_extracted.tsv"

# List of books
books = [
    "The First Book of Moses: Called Genesis", "The Second Book of Moses: Called Exodus",
    "The Third Book of Moses: Called Leviticus", "The Fourth Book of Moses: Called Numbers",
    "The Fifth Book of Moses: Called Deuteronomy", "The Book of Joshua", "The Book of Judges",
    "The Book of Ruth", "The First Book of Samuel", "The Second Book of Samuel",
    "The First Book of the Kings", "The Second Book of the Kings", "The First Book of the Chronicles",
    "The Second Book of the Chronicles", "Ezra", "The Book of Nehemiah", "The Book of Esther",
    "The Book of Job", "The Book of Psalms", "The Proverbs", "Ecclesiastes",
    "The Song of Solomon", "The Book of the Prophet Isaiah", "The Book of the Prophet Jeremiah",
    "The Lamentations of Jeremiah", "The Book of the Prophet Ezekiel", "The Book of Daniel",
    "Hosea", "Joel", "Amos", "Obadiah", "Jonah", "Micah", "Nahum", "Habakkuk", "Zephaniah",
    "Haggai", "Zechariah", "Malachi", "The Gospel According to Saint Matthew",
    "The Gospel According to Saint Mark", "The Gospel According to Saint Luke",
    "The Gospel According to Saint John", "The Acts of the Apostles",
    "The Epistle of Paul the Apostle to the Romans", "The First Epistle of Paul the Apostle to the Corinthians",
    "The Second Epistle of Paul the Apostle to the Corinthians",
    "The Epistle of Paul the Apostle to the Galatians", "The Epistle of Paul the Apostle to the Ephesians",
    "The Epistle of Paul the Apostle to the Philippians", "The Epistle of Paul the Apostle to the Colossians",
    "The First Epistle of Paul the Apostle to the Thessalonians",
    "The Second Epistle of Paul the Apostle to the Thessalonians",
    "The First Epistle of Paul the Apostle to Timothy", "The Second Epistle of Paul the Apostle to Timothy",
    "The Epistle of Paul the Apostle to Titus", "The Epistle of Paul the Apostle to Philemon",
    "The Epistle of Paul the Apostle to the Hebrews", "The General Epistle of James",
    "The First Epistle General of Peter", "The Second General Epistle of Peter",
    "The First Epistle General of John", "The Second Epistle General of John",
    "The Third Epistle General of John", "The General Epistle of Jude",
    "The Revelation of Saint John the Divine"
]

# Regular expression patterns
book_pattern = re.compile(r"^(" + "|".join(re.escape(book) for book in books) + r")$")
verse_pattern = re.compile(r"^(\d+):(\d+)\s(.+)$")

# Initialize variables
current_book = None
current_chapter = None
current_verse = None
current_text = []
data = []

# Read the file and process
with open(input_file, "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()

        # Check for book title
        if book_pattern.match(line):
            # If there is ongoing text, save it before switching books
            if current_book and current_text:
                data.append([current_book, current_chapter, current_verse, " ".join(current_text)])
            current_book = line
            current_chapter = None
            current_verse = None
            current_text = []
            continue

        # Check for chapter and verse
        match = verse_pattern.match(line)
        if match:
            # If there is ongoing text, save it before starting a new verse
            if current_book and current_text:
                data.append([current_book, current_chapter, current_verse, " ".join(current_text)])
            current_chapter, current_verse, text = match.groups()
            current_chapter = int(current_chapter)
            current_verse = int(current_verse)
            current_text = [text]
        elif current_text:
            # Append additional lines to the current verse
            current_text.append(line)

# Save the last ongoing verse
if current_book and current_text:
    data.append([current_book, current_chapter, current_verse, " ".join(current_text)])

# Create DataFrame and save
df = pd.DataFrame(data, columns=["Book", "Chapter", "Verse", "Text"])
df.to_csv(output_file, sep="\t", index=False)
print(f"Extraction complete. Data saved to {output_file}.")



In [ ]:
# Load the extracted file
df = pd.read_csv("KJV_extracted.tsv", sep="\t")

# Count unique books
unique_books = df["Book"].nunique()
print(f"Unique books extracted: {unique_books}")

# Compare with expected list
missing_books = [book for book in books if book not in df["Book"].unique()]
print(f"Missing books: {missing_books}")

# OEB

In [ ]:
import re

# File paths
input_file = 'OEB.txt'
output_tsv_file = 'OEB_combined.tsv'

# Book names extracted from the table of contents
book_names = [
    "Ruth", "Esther", "Psalms", "Hosea", "Joel", "Amos", "Obadiah", "Jonah", 
    "Micah", "Nahum", "Habakkuk", "Zephaniah", "Haggai", "Zechariah", "Malachi",
    "Matthew", "Mark", "Luke", "John", "Acts", "Romans", "1 Corinthians", 
    "2 Corinthians", "Galatians", "Ephesians", "Philippians", "Colossians", 
    "1 Thessalonians", "2 Thessalonians", "1 Timothy", "2 Timothy", "Titus", 
    "Philemon", "Hebrews", "James", "1 Peter", "2 Peter", "1 John", "2 John", 
    "3 John", "Jude", "Revelation"
]

# Regex pattern to detect verses
verse_pattern = re.compile(r"\[(\d+:\d+)\]\s+(.+?)(?=\[\d+:\d+\]|$)")

# Data collection
combined_data = []
current_book = None

# Use the order of books from the table of contents to infer context
book_index = 0

with open(input_file, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()

        # Detect verses
        for verse_match in verse_pattern.finditer(line):
            verse_id, verse_text = verse_match.groups()
            chapter_num, verse_num = verse_id.split(":")

            # Assign current book based on the detected order in the text
            if current_book is None or len(combined_data) > 0 and int(chapter_num) == 1 and int(verse_num) == 1:
                current_book = book_names[book_index]
                book_index += 1

            # Collect data
            combined_data.append((current_book, chapter_num, verse_num, verse_text.strip()))

# Write the data to the TSV file
with open(output_tsv_file, 'w', encoding='utf-8') as out_file:
    out_file.write("Book\tChapter\tVerse\tText\n")
    for entry in combined_data:
        out_file.write("\t".join(entry) + "\n")

print(f"Extraction complete! {len(combined_data)} verses written to {output_tsv_file}")


# WEB

In [ ]:
import re

# File paths for input and output
input_web_file = "WEB.txt"
output_web_tsv_file = "WEB_combined.tsv"

# Regex patterns for books, chapters, and verses
book_pattern = re.compile(r"^Book \d+\s+(.+)$")  # Matches lines like "Book 01 Genesis"
verse_pattern = re.compile(r"^(\d{3}):(\d{3})\s+(.+)$")  # Matches lines like "001:001 Text"

# Function to normalize chapter and verse numbers
def normalize_number(value):
    return str(int(value))  # Remove leading zeros by converting to integer and back to string

# Function to clean text
def clean_text(text):
    return re.sub(r'[\"“”]', '', text).strip()  # Remove quotation marks and clean whitespace

# Data collection
web_data = []
current_book = None
current_verse = None
current_text = []

with open(input_web_file, "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()

        # Detect book titles
        book_match = book_pattern.match(line)
        if book_match:
            current_book = book_match.group(1).strip()
            continue

        # Detect verses
        verse_match = verse_pattern.match(line)
        if verse_match and current_book:
            # Save the previous verse if it exists
            if current_verse and current_text:
                full_text = " ".join(current_text).strip()
                full_text = clean_text(full_text)  # Clean the text
                web_data.append((current_book, *current_verse, full_text))

            # Start a new verse
            chapter_num, verse_num, text = verse_match.groups()
            chapter_num = normalize_number(chapter_num)  # Normalize chapter number
            verse_num = normalize_number(verse_num)      # Normalize verse number
            current_verse = (chapter_num, verse_num)
            current_text = [text]
        else:
            # Accumulate lines for the current verse
            current_text.append(line)

    # Save the last verse
    if current_verse and current_text:
        full_text = " ".join(current_text).strip()
        full_text = clean_text(full_text)  # Clean the text
        web_data.append((current_book, *current_verse, full_text))

# Write data to a single TSV file
with open(output_web_tsv_file, "w", encoding="utf-8") as out_file:
    out_file.write("Book\tChapter\tVerse\tText\n")
    for entry in web_data:
        out_file.write("\t".join(entry) + "\n")

print(f"Processed {len(web_data)} entries. Cleaned WEB file saved as {output_web_tsv_file}.")

## DRB

In [ ]:
DRB_old_test_books = """The Book of Genesis
 The Book of Exodus
 The Book of Leviticus
 The Book of Numbers
 The Book of Deuteronomy
 The Book of Josue
 The Book of Judges
 The Book of Ruth
 The First Book of Samuel, otherwise called the First Book of Kings
 The Second Book of Samuel, otherwise called the Second Book of Kings
 The Third Book of Kings
 The Fourth Book of Kings
 The First Book of Paralipomenon
 The Second Book of Paralipomenon
 The First Book of Esdras
 The Book of Nehemias, which is called the Second of Esdras
 The Book of Tobias
 The Book of Judith
 The Book of Esther
 The Book of Job
 The Book of Psalms
 The Book of Proverbs
 Ecclesiastes
 Solomon’s Canticle of Canticles
 The Book of Wisdom
 Ecclesiasticus
 The Prophecy of Isaias
 The Prophecy of Jeremias
 The Lamentations of Jeremias
 The Prophecy of Baruch
 The Prophecy of Ezechiel
 The Prophecy of Daniel
 The Prophecy of Osee
 The Prophecy of Joel
 The Prophecy of Amos
 The Prophecy of Abdias
 The Prophecy of Jonas
 The Prophecy of Micheas
 The Prophecy of Nahum
 The Prophecy of Habacuc
 The Prophecy of Sophonias
 The Prophecy of Aggeus
 The Prophecy of Zacharias
 The Prophecy of Malachias
 The First Book of Machabees
 The Second Book of Machabees""".split('\n ')

DRB_new_test_books = """The Holy Gospel of Jesus Christ According to St. Matthew
 The Holy Gospel of Jesus Christ According to St. Mark
 The Holy Gospel of Jesus Christ According to St. Luke
 The Holy Gospel of Jesus Christ  According to St. John
 The Acts of the Apostles
 The Epistle of St. Paul the Apostle to the Romans
 The First Epistle of St. Paul to the Corinthians
 The Second Epistle of St. Paul to the Corinthians
 The Epistle of St. Paul to the Galatians
 The Epistle of St. Paul to the Ephesians
 The Epistle of St. Paul to the Philippians
 The Epistle of St. Paul to the Colossians
 The First Epistle of St. Paul to the Thessalonians
 The Second Epistle of St. Paul to the Thessalonians
 The First Epistle of St. Paul to Timothy
 The Second Epistle of St. Paul to Timothy
 The Epistle of St. Paul to Titus
 The Epistle of St. Paul to Philemon
 The Epistle of St. Paul to the Hebrews
 The Catholic Epistle of St. James the Apostle
 The First Epistle of St. Peter the Apostle
 The Second Epistle of St. Peter the Apostle
 The First Epistle of St. John the Apostle
 The Second Epistle of St. John the Apostle
 The Third Epistle of St. John the Apostle
 The Catholic Epistle of St. Jude the Apostle
 The Apocalypse of St. John the Apostle""".split('\n ')
print(DRB_old_test_books)
print(DRB_new_test_books)

In [ ]:
txt = "The Book of Leviticus"
print(txt.upper())

In [ ]:
import re

# List of book titles (assuming they are already in uppercase)
titles = DRB_old_test_books + DRB_new_test_books
titles = [element.upper() for element in titles]

# Open the input Bible text file
with open('DRV.txt', "r") as f:
    line_counter = 0
    book_name = None
    book_content = ""  # Temporary storage for the current book's content

    # Iterate through each line in the file
    for line in f:
        line_counter += 1
        
        # Only process lines between 145 and 140345
        if line_counter < 145:
            continue  # Skip lines before 145
        if line_counter > 140345:
            break  # Stop processing after line 140345

        line = line.strip()  # Remove leading and trailing whitespace
        
        # Check if the line contains a book title
        for title in titles:
            if title in line:  # If a book title is found
                if book_name:  # Process the previous book if it exists
                    # Save the content of the previous book into a text file
                    with open(f'DRB_{book_name}.txt', 'w') as book_file:
                        book_file.write(book_content)
                
                # Set the new book name and reset content for the new book
                book_name = title
                book_content = ""  # Reset content for the next book
                break  # Stop checking for other titles once the current one is found

        # Append the current line to the book's content (if book_name is set)
        if book_name:
            book_content += line + "\n"  # Add newline between lines of the book

    # Save the last book's content after processing all lines
    if book_name and book_content:
        with open(f'DRB_{book_name}.txt', 'w') as book_file:
            book_file.write(book_content)

print("Books have been saved to separate text files.")


In [141]:
import pandas as pd

# File paths for the filtered TSV files
file_paths = {
    "DRB": "DRB_filtered_with_index.tsv",
    "KJV": "KJV_filtered_with_index.tsv",
    "OEB": "OEB_filtered_with_index.tsv",
    "WEB": "WEB_filtered_with_index.tsv"
}

# List to hold DataFrames for each version
dataframes = []

# Add a "Version" column to each TSV file and load it
for version, path in file_paths.items():
    try:
        df = pd.read_csv(path, sep="\t")
        df["Version"] = version  # Add version column
        dataframes.append(df)
    except Exception as e:
        print(f"Error processing {version}: {e}")

# Combine all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Ensure Chapter and Verse columns are integers for proper sorting
combined_df["Chapter"] = combined_df["Chapter"].astype(int)
combined_df["Verse"] = combined_df["Verse"].astype(int)

# Sort by Book, then by Version, Chapter, and Verse
combined_df = combined_df.sort_values(by=["Book", "Version", "Chapter", "Verse"])

# Reorder columns for readability
column_order = ["Index", "Version", "Book", "Chapter", "Verse", "Text"]
combined_df = combined_df[column_order]

# Save to a new TSV file
output_file = "aligned_bible_final.tsv"
combined_df.to_csv(output_file, sep="\t", index=False)

print(f"Combined file grouped by Book saved to: {output_file}")



Combined file grouped by Book saved to: aligned_bible_final.tsv


## sorted alignment

In [142]:
import pandas as pd

# Load the combined data
combined_file = "aligned_bible_final.tsv"
df = pd.read_csv(combined_file, sep="\t")

# Ensure Chapter and Verse are treated as integers
df["Chapter"] = df["Chapter"].astype(int, errors="ignore")
df["Verse"] = df["Verse"].astype(int, errors="ignore")

# Sort the DataFrame
df = df.sort_values(
    by=["Index", "Version", "Book", "Chapter", "Verse"],  # Sorting keys
)

# Save to a new file
output_file = "sorted_aligned_bible_final.tsv"
df.to_csv(output_file, sep="\t", index=False)

print(f"Sorted combined data saved to: {output_file}")



Sorted combined data saved to: sorted_aligned_bible_final.tsv


In [ ]:
import pandas as pd
import re

# File paths
input_file = "DRB_preprocessed_columns.tsv"
output_file = "DRB_normalized_titles.tsv"

# List of canonical book titles
canonical_titles = DRB_old_test_books + DRB_new_test_books

# Function to normalize a title
def normalize_title(title):
    # Convert title to lowercase and strip whitespace
    title = title.strip().lower()
    # Match with canonical titles (case insensitive)
    for canonical in canonical_titles:
        if re.fullmatch(canonical.strip().lower(), title):
            return canonical  # Return the canonical title if it matches
    return title  # Return original title if no match is found

# Read the TSV file
df = pd.read_csv(input_file, sep="\t")

# Normalize the "Book" column
df["Book"] = df["Book"].apply(normalize_title)

# Save the updated DataFrame to a new TSV file
df.to_csv(output_file, sep="\t", index=False)

print(f"Book titles in '{input_file}' have been normalized and saved to '{output_file}'.")

## fix alignment file

In [145]:
import pandas as pd
import re

# File paths
aligned_file = "def_titles_aligned.csv"  # Path to aligned titles file
file_paths = {
    "OEB": "OEB_combined.tsv",
    "DRB": "DRB_normalized_titles.tsv",
    "KJV": "KJV_final.tsv",
    "WEB": "WEB_combined.tsv"
}

# Function to normalize titles
def normalize_title(title):
    if not isinstance(title, str):
        return title
    title = re.sub(r'\b1\b', 'first', title, flags=re.IGNORECASE)
    title = re.sub(r'\b2\b', 'second', title, flags=re.IGNORECASE)
    title = re.sub(r'\b3\b', 'third', title, flags=re.IGNORECASE)
    title = re.sub(r'\bthe\b', '', title, flags=re.IGNORECASE)
    title = re.sub(r'\bepistle of\b', 'epistle', title, flags=re.IGNORECASE)
    return title.lower().strip()

# Step 1: Load the aligned titles file
aligned_df = pd.read_csv(aligned_file)

# Add an index to the aligned titles for sorting purposes
aligned_df["Index"] = aligned_df.index + 1  # Start index from 1 for readability

# Step 2: Create a mapping of normalized titles to actual titles for each version
title_mappings = {}
unmatched_titles = {version: [] for version in file_paths.keys()}  # Log unmatched titles

for version, path in file_paths.items():
    try:
        # Load the TSV file
        df = pd.read_csv(path, sep="\t")
        # Normalize the titles and create a mapping
        df["Normalized Book"] = df["Book"].apply(normalize_title)
        title_mappings[version] = dict(zip(df["Normalized Book"], df["Book"]))
    except Exception as e:
        print(f"Error processing {version}: {e}")
        title_mappings[version] = {}

# Step 3: Filter each version based on the aligned titles
for version, path in file_paths.items():
    try:
        # Load the respective TSV file
        version_df = pd.read_csv(path, sep="\t")
        
        # Prepare the filtered DataFrame
        filtered_rows = []

        # Iterate over each row in the alignment file
        for _, row in aligned_df.iterrows():
            aligned_title = row[version]  # Get the aligned title for this version
            
            # Find the corresponding actual title from the mapping
            actual_title = title_mappings[version].get(normalize_title(aligned_title), None)
            if actual_title:
                # Filter rows matching the actual title
                matched_rows = version_df[version_df["Book"] == actual_title].copy()
                # Add Index and Version columns
                matched_rows["Index"] = row["Index"]
                matched_rows["Version"] = version  
                filtered_rows.append(matched_rows)

        # Combine all filtered rows
        if filtered_rows:
            filtered_version_df = pd.concat(filtered_rows, ignore_index=True)
        else:
            filtered_version_df = pd.DataFrame(columns=version_df.columns.tolist())

        # Save the filtered DataFrame
        output_file = f"{version}_filtered_with_index.tsv"
        filtered_version_df.to_csv(output_file, sep="\t", index=False)
    except Exception as e:
        print(f"Error processing {version}: {e}")

In [146]:
# Combine all filtered versions
filtered_files = ["OEB_filtered_with_index.tsv", "DRB_filtered_with_index.tsv", "KJV_filtered_with_index.tsv", "WEB_filtered_with_index.tsv"]
combined_df = pd.concat([pd.read_csv(file, sep="\t") for file in filtered_files], ignore_index=True)

# Ensure Chapter and Verse columns are integers for sorting
combined_df["Chapter"] = combined_df["Chapter"].astype(int, errors="ignore")
combined_df["Verse"] = combined_df["Verse"].astype(int, errors="ignore")

# Sort by Index, Version, Chapter, and Verse
combined_df = combined_df.sort_values(by=["Index", "Version", "Chapter", "Verse"])

# Reorder columns
reorder_columns = ["Index", "Version", "Book", "Chapter", "Verse", "Text"]
combined_df = combined_df[reorder_columns]  # Reorder columns

# Save the final sorted and reordered file
output_file = "sorted_aligned_bible_final.tsv"
combined_df.to_csv(output_file, sep="\t", index=False)

print(f"Final sorted and reordered file saved to: {output_file}")


Final sorted and reordered file saved to: sorted_aligned_bible_final.tsv


In [147]:
# inspect missing KJV books

import pandas as pd

# Load the final sorted file
df = pd.read_csv("sorted_aligned_bible_final.tsv", sep="\t")

# Check for missing or empty entries in the Version column
print("Missing or Empty Version Entries:")
print(df[df["Version"].isnull() | (df["Version"] == "")].head())

# Check unique versions present in the file
print("Unique Versions in File:")
print(df["Version"].unique())

# Filter for KJV-related rows and check for missing books
kjv_data = df[df["Version"] == "KJV"]
missing_kjv_books = kjv_data[kjv_data["Book"].isnull() | (kjv_data["Book"] == "")]
print("Missing KJV Books:")
print(missing_kjv_books.head())

# Summarize insights
print(f"Total Rows: {len(df)}")
print(f"Rows with Missing Version: {len(df[df['Version'].isnull() | (df['Version'] == '')])}")
print(f"Total KJV Rows: {len(kjv_data)}")
print(f"KJV Rows with Missing Books: {len(missing_kjv_books)}")


Missing or Empty Version Entries:
Empty DataFrame
Columns: [Index, Version, Book, Chapter, Verse, Text]
Index: []
Unique Versions in File:
['DRB' 'KJV' 'OEB' 'WEB']
Missing KJV Books:
Empty DataFrame
Columns: [Index, Version, Book, Chapter, Verse, Text]
Index: []
Total Rows: 25041
Rows with Missing Version: 0
Total KJV Rows: 5895
KJV Rows with Missing Books: 0


In [148]:
import pandas as pd

# List of filtered file paths
filtered_files = [
    "OEB_filtered_with_index.tsv",
    "DRB_filtered_with_index.tsv",
    "KJV_filtered_with_index.tsv",
    "WEB_filtered_with_index.tsv"
]

# Function to count unique book titles in a file
def count_unique_titles(file_path):
    try:
        df = pd.read_csv(file_path, sep="\t")  # Load the filtered file
        unique_titles = df["Book"].nunique()  # Count unique titles
        return unique_titles
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Count unique titles in each file
unique_title_counts = {file: count_unique_titles(file) for file in filtered_files}

# Display the results
print(unique_title_counts)



{'OEB_filtered_with_index.tsv': 25, 'DRB_filtered_with_index.tsv': 24, 'KJV_filtered_with_index.tsv': 24, 'WEB_filtered_with_index.tsv': 24}


In [149]:
# Load the extracted file
df = pd.read_csv("sorted_aligned_bible_final.tsv", sep="\t")

# Count unique books
unique_books = df["Book"].nunique()
print(f"Unique books extracted: {unique_books}")

Unique books extracted: 70
